In [ ]:
import argparse
import logging
import os
import sys

import boto3
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from typing import Optional
import requests
import json

ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
sys.path.insert(0, os.path.join(ROOT_DIR, 'src'))

# import IIIFingest
from IIIFingest.auth import Credentials  # noqa: E402
from IIIFingest.client import Client  # noqa: E402
from IIIFingest.ingest import signed_request, endpoint_to_proxy

In [ ]:
proxy = "https://PROXY_HERE.lambda-url.us-east-1.on.aws"

In [ ]:
def test_job_status(
    endpoint="https://mps-admin-dev.lib.harvard.edu/admin/ingest/jobstatus/6255a8a966b08c163ac5c880/",
    proxy=proxy
    ):
    # Logging configuration
    root = logging.getLogger()
    root.setLevel(logging.INFO)

    pkg = logging.getLogger('IIIFingest')
    pkg.setLevel(logging.DEBUG)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    handler.setFormatter(formatter)
    if (root.hasHandlers()):
        root.handlers.clear()
    root.addHandler(handler)
    
    # be signed in with tlt-dev admin
    tlt_session = boto3.Session()
    tlt_credentials = tlt_session.get_credentials()
    tlt_creds = tlt_credentials.get_frozen_credentials()
    
    # try the proxied endpoint
    proxied_endpoint = endpoint_to_proxy(endpoint, proxy)
    print(proxied_endpoint)
        
    signed = signed_request(
        method="GET",
        url=proxied_endpoint,
        creds=tlt_creds,
        headers={"Content-Type": "application/x-amz-json-1.1"}
    )
    return(signed)

In [ ]:
s = test_job_status()
s.json()

In [ ]:
def test_ingest_pipeline(
    proxy=None,
    asset_prefix = "mappingcolor",
    issuer = "atmch",
    space = "atmch",
    environment = "qa"
) -> None:

    # Logging configuration
    
    root = logging.getLogger()
    root.setLevel(logging.INFO)

    pkg = logging.getLogger('IIIFingest')
    pkg.setLevel(logging.DEBUG)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    handler.setFormatter(formatter)
    if (root.hasHandlers()):
        root.handlers.clear()
    root.addHandler(handler)
    
    # be signed in with tlt-dev admin
    tlt_session = boto3.Session()
    tlt_credentials = tlt_session.get_credentials()
    tlt_creds = tlt_credentials.get_frozen_credentials()

    # Client configuration
    # asset_prefix = args.asset_prefix
    # issuer = args.issuer
    # space = args.space
    # environment = args.environment
    session = boto3.Session(profile_name=f"mps-{space}-{environment}")

    jwt_creds = Credentials(
        issuer=issuer,
        kid=f"{issuer}default",
        private_key_path=os.path.join(
            ROOT_DIR, f"auth/{environment}/keys/{issuer}/{issuer}default/private.key"
        ),
    )

    client = Client(
        space=space,
        environment=environment,
        # asset_prefix=asset_prefix,
        jwt_creds=jwt_creds,
        boto_session=session,
        proxy=proxy,  # Add proxy here if not on VPN
    )

    images = [
        {
            "label": "27.586.126A",
            "filepath": os.path.join(ROOT_DIR, "tests/images/mcihtest1.tif"),
        },
        {
            "label": "27.586.248A",
            "filepath": os.path.join(ROOT_DIR, "tests/images/mcihtest2.tif"),
            "metadata": [{"label": "Test", "value": "Image level metadata"}],
        },
        {
            "label": "27.586.249A",
            "filepath": os.path.join(ROOT_DIR, "tests/images/mcihtest3.tif"),
        },
    ]

    manifest_level_metadata = {
        "labels": ["Test Manifest MCIH"],
        "metadata": [
            {
                "label": "Creator",
                "value": "Unknown",
                "label_lang": "en",
                "value_lang": "en",
            },
            {
                "label": "Date",
                "value": "19th Century",
                "label_lang": "en",
                "value_lang": "en",
            },
        ],
        "required_statement": [{"label": "Attribution", "value": "Jinah Kim"}],
        "default_lang": "en",
        "service_type": "ImageService2",
        "service_profile": "level2",
        "rights": "http://creativecommons.org/licenses/by-sa/3.0/",
        "summary": "A test manifest for Mapping Color in History ingest into MPS IIIF delivery solution",
        "providers": [
            {
                "labels": [
                    {
                        "lang": "en",
                        "value": "Harvard University - Arts and Humanities Research Computing (organizing org)",
                    }
                ],
                "id": "http://id.loc.gov/authorities/names/n78096930",
            },
            {
                "labels": [{"value": "Harvard Art Museum (providing org)"}],
                "id": "http://id.loc.gov/authorities/names/no2008065752",
            },
        ],
    }

    assets = client.upload(images=images, s3_path="images/")

    manifest = client.create_manifest(
        manifest_level_metadata=manifest_level_metadata,
        assets=assets,
    )

    result = client.ingest(
        assets=assets,
        manifest=manifest,
    )

    status = client.jobstatus(result["job_id"])
    print("Done: ", status)


In [ ]:
test_ingest_pipeline(proxy=proxy)

In [ ]:
tlt_session = boto3.Session()
tlt_credentials = tlt_session.get_credentials()
tlt_creds = tlt_credentials.get_frozen_credentials()

signed = signed_request(
    method="GET",
    url=f"{proxy}/dev/jobstatus/6255a8a966b08c163ac5c880/",
    creds=tlt_creds,
    headers={"Content-Type": "application/x-amz-json-1.1"}
)
print(signed.json())

In [ ]:
ingest_data = {'globalSettings': {'actionDefault': 'upsert', 'spaceDefault': 'atmch'}, 'metadata': {}, 'assets': {'audio': [], 'video': [], 'text': [], 'image': [{'action': 'create', 'storageSrcPath': 'images/', 'storageSrcKey': 'mcihtest1.tif', 'identifier': 'AT:5GFuCExWrEewNecqfhZKPe', 'space': 'atmch', 'createdByAgent': 'atagent', 'createDate': '2022-04-25 12:36:19', 'lastModifiedByAgent': 'atagent', 'lastModifiedDate': '2022-04-25 12:36:19', 'status': 'ACTIVE', 'iiifApiVersion': '3', 'policyDefinition': {'policyGroupName': 'default'}, 'assetMetadata': [{'fieldName': 'imageSize', 'jsonValue': {'width': 3600, 'height': 584}}]}, {'action': 'create', 'storageSrcPath': 'images/', 'storageSrcKey': 'mcihtest2.tif', 'identifier': 'AT:hmi96YrbNWzcvxNqAgmZg3', 'space': 'atmch', 'createdByAgent': 'atagent', 'createDate': '2022-04-25 12:36:19', 'lastModifiedByAgent': 'atagent', 'lastModifiedDate': '2022-04-25 12:36:19', 'status': 'ACTIVE', 'iiifApiVersion': '3', 'policyDefinition': {'policyGroupName': 'default'}, 'assetMetadata': [{'fieldName': 'imageSize', 'jsonValue': {'width': 3600, 'height': 621}}]}, {'action': 'create', 'storageSrcPath': 'images/', 'storageSrcKey': 'mcihtest3.tif', 'identifier': 'AT:Dw6EKUzv4H8cFQg6Eq5RKZ', 'space': 'atmch', 'createdByAgent': 'atagent', 'createDate': '2022-04-25 12:36:19', 'lastModifiedByAgent': 'atagent', 'lastModifiedDate': '2022-04-25 12:36:19', 'status': 'ACTIVE', 'iiifApiVersion': '3', 'policyDefinition': {'policyGroupName': 'default'}, 'assetMetadata': [{'fieldName': 'imageSize', 'jsonValue': {'width': 3600, 'height': 621}}]}]}, 'manifest': {'@context': 'http://iiif.io/api/presentation/3/context.json', 'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3', 'type': 'Manifest', 'label': {'en': ['Test Manifest MCIH']}, 'metadata': [{'label': {'en': ['Creator']}, 'value': {'en': ['Unknown']}}, {'label': {'en': ['Date']}, 'value': {'en': ['19th Century']}}], 'requiredStatement': {'label': {'en': ['Attribution']}, 'value': {'en': ['Jinah Kim']}}, 'rights': 'http://creativecommons.org/licenses/by-sa/3.0/', 'behavior': ['paged'], 'provider': [{'id': 'http://id.loc.gov/authorities/names/n78096930', 'type': 'Agent', 'label': {'en': ['Harvard University - Arts and Humanities Research Computing (organizing org)']}}, {'id': 'http://id.loc.gov/authorities/names/no2008065752', 'type': 'Agent', 'label': {'en': ['Harvard Art Museum (providing org)']}}], 'items': [{'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/canvas/canvas:5GFuCExWrEewNecqfhZKPe', 'type': 'Canvas', 'label': {'en': ['5GFuCExWrEewNecqfhZKPe']}, 'height': 584, 'width': 3600, 'items': [{'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/annotationPage/annopage:5GFuCExWrEewNecqfhZKPe', 'type': 'AnnotationPage', 'items': [{'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/annotation/annotation:5GFuCExWrEewNecqfhZKPe', 'type': 'Annotation', 'motivation': 'painting', 'body': {'id': 'https://mps-qa.lib.harvard.edu/assets/images/AT:5GFuCExWrEewNecqfhZKPe/full/max/0/default.tiff', 'type': 'Image', 'format': 'image/tiff', 'height': 584, 'width': 3600, 'service': [{'id': 'https://mps-qa.lib.harvard.edu/assets/images/AT:5GFuCExWrEewNecqfhZKPe', 'type': 'ImageService2', 'profile': 'level2'}]}, 'target': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/canvas/canvas:5GFuCExWrEewNecqfhZKPe'}]}]}, {'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/canvas/canvas:hmi96YrbNWzcvxNqAgmZg3', 'type': 'Canvas', 'label': {'en': ['hmi96YrbNWzcvxNqAgmZg3']}, 'height': 621, 'width': 3600, 'items': [{'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/annotationPage/annopage:hmi96YrbNWzcvxNqAgmZg3', 'type': 'AnnotationPage', 'items': [{'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/annotation/annotation:hmi96YrbNWzcvxNqAgmZg3', 'type': 'Annotation', 'motivation': 'painting', 'body': {'id': 'https://mps-qa.lib.harvard.edu/assets/images/AT:hmi96YrbNWzcvxNqAgmZg3/full/max/0/default.tiff', 'type': 'Image', 'format': 'image/tiff', 'height': 621, 'width': 3600, 'service': [{'id': 'https://mps-qa.lib.harvard.edu/assets/images/AT:hmi96YrbNWzcvxNqAgmZg3', 'type': 'ImageService2', 'profile': 'level2'}]}, 'target': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/canvas/canvas:hmi96YrbNWzcvxNqAgmZg3'}]}]}, {'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/canvas/canvas:Dw6EKUzv4H8cFQg6Eq5RKZ', 'type': 'Canvas', 'label': {'en': ['Dw6EKUzv4H8cFQg6Eq5RKZ']}, 'height': 621, 'width': 3600, 'items': [{'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/annotationPage/annopage:Dw6EKUzv4H8cFQg6Eq5RKZ', 'type': 'AnnotationPage', 'items': [{'id': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/annotation/annotation:Dw6EKUzv4H8cFQg6Eq5RKZ', 'type': 'Annotation', 'motivation': 'painting', 'body': {'id': 'https://mps-qa.lib.harvard.edu/assets/images/AT:Dw6EKUzv4H8cFQg6Eq5RKZ/full/max/0/default.tiff', 'type': 'Image', 'format': 'image/tiff', 'height': 621, 'width': 3600, 'service': [{'id': 'https://mps-qa.lib.harvard.edu/assets/images/AT:Dw6EKUzv4H8cFQg6Eq5RKZ', 'type': 'ImageService2', 'profile': 'level2'}]}, 'target': 'https://nrs-qa.lib.harvard.edu/URN-3:AT:GENJQDmHuAeKC57iEbJE9Sjsc:MANIFEST:3/canvas/canvas:Dw6EKUzv4H8cFQg6Eq5RKZ'}]}]}]}}

In [ ]:
def signed_request(
    method: str,
    url: str,
    creds,  # AWS credentials
    region: str = "us-east-1",
    service_name: str = "lambda",
    data: Optional[dict] = None,
    params: Optional[dict] = None,
    headers: Optional[dict] = None,
):
    """Sign requests which will be proxied to the ingest endpoint through AWS Lambda function URLs"""
    request = AWSRequest(
        method=method, url=url, data=data, params=params, headers=headers
    )
    # "service_name" is generally "execute-api" for signing API Gateway requests
    SigV4Auth(creds, service_name, region).add_auth(request)
    return requests.request(
        method=method,
        url=url,
        headers=dict(request.headers),
        data=data,
        # json = data
    )

In [ ]:
issuer = "atmch"
environment="qa"
jwt_creds = Credentials(
    issuer=issuer,
    kid=f"{issuer}default",
    private_key_path=os.path.join(
        ROOT_DIR, f"auth/{environment}/keys/{issuer}/{issuer}default/private.key"
    ),
)
token = jwt_creds.make_jwt()

In [ ]:
tlt_session = boto3.Session()
tlt_credentials = tlt_session.get_credentials()
tlt_creds = tlt_credentials.get_frozen_credentials()

# # If `json` param is passed instead of `data` in the signed_request method, it overrides it and sets the content type to application/json,
# which I think conflicts with headers={"Content-Type": "application/x-amz-json-1.1"}
# This causes an auth failure: The request signature we calculated does not match the signature you provided. Check your AWS Secret Access Key and signing method. Consult the service documentation for details.
# But if we send it as data, it doesn't include everything - the whole dictionary is missing a lot of things
# gets posted as base64 and looks like this:
# b'globalSettings=actionDefault&globalSettings=spaceDefault&assets=audio&assets=video&assets=text&assets=image&manifest=%40context&manifest=id&manifest=type&manifest=label&manifest=metadata&manifest=requiredStatement&manifest=rights&manifest=behavior&manifest=provider&manifest=items'

data = {
    "payload": ingest_data,
    "token": f"Bearer {token}"
}
json_string = json.dumps(data)

signed = signed_request(
    method="POST",
    url=f"{proxy}/qa/initialize/",
    creds=tlt_creds,
    # headers={"Content-Type": "application/json"},
    headers={
        "Content-Type": "application/x-amz-json-1.1",
        "Authorization": f"Bearer {token}",
    }, 
    # headers={"Content-Type": "application/x-www-form-urlencoded"},
    # data=ingest_data
    data=json_string
)
print(data)
print(signed.request.headers)
from urllib.parse import parse_qsl
print(dict(parse_qsl(signed.request.body)))
print(signed.text)
# print(signed.request.data)
print(signed.json())

In [ ]:
tlt_creds